In [1]:
from typing import Any, Callable, Dict, Optional
from numpy import ndarray
from scipy.sparse import csc_matrix
from typing import Union
from typing import Optional
from warnings import warn
from numpy import hstack, ndarray
from numpy.linalg import norm
from scipy import sparse
from scs import solve
import numpy as np


In [2]:
DenseOrCSCMatrix = Union[ndarray, csc_matrix]

def scs_solve_qp( P, q, G, h, A, b, initvals: Optional[ndarray] = None, 
                 eps_abs: float = 1e-7, eps_rel: float = 1e-7, verbose: 
                 bool = False, **kwargs,) -> Optional[ndarray]:

    if isinstance(P, ndarray):
        P = sparse.csc_matrix(P)
    if isinstance(G, ndarray):
        G = sparse.csc_matrix(G)
    if isinstance(A, ndarray):
        A = sparse.csc_matrix(A)
    kwargs.update({ "eps_abs": eps_abs,"eps_rel": eps_rel,"verbose": verbose,})
    
    data = {"P": P, "c": q}
    cone = {}
    if initvals is not None:
        data["x"] = initvals
    
    data["A"] = sparse.vstack([A, G], format="csc")
    data["b"] = hstack([b, h])
    cone["z"] = b.shape[0]  # zero cone
    cone["l"] = h.shape[0]  # positive orthant
   
    solution = solve(data, cone, **kwargs)
    status_val = solution["info"]["status_val"]
    if status_val != 1:
        warn(
            f"SCS returned {status_val}: {__status_val_meaning__[status_val]}"
        )
        if status_val != 2:
            # solution is not inaccurate, so the optimization failed
            return None
    return solution["x"]

In [3]:
m = 5
n = 700
p = 5
P = np.loadtxt('P.txt')
q = np.loadtxt('q.txt')
G = np.loadtxt('G.txt')
h = np.loadtxt('h.txt')
A = np.loadtxt('A.txt')
b = np.loadtxt('b.txt')


x=scs_solve_qp( P,q, G,h,A,b)
v=0.5*x.T@P@x+q.T@x
print(v)

-42.31408315686017
